Задание 1: Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.
Задание 2: Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.
Задание 3: Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).
Задание 4: Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).
Задание 5: Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.
Задание 6: Могла ли модель переобучиться? Почему?
Задание 7 (опциональное): Создайте функции eval_model_l1 и eval_model_l2 с применением L1 и L2 регуляризаций соответственно.

In [20]:
import numpy as np
import matplotlib.pyplot as plt

In [21]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450,
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 
               1, 3, 3, 1, 2]], dtype = np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [31]:
def sigmoid(W, X):
    z = np.dot(W, X)
    sg = 1 / (1 + np.exp(-z))
    return sg

# функция logloss
def calc_logloss(y, y_pred):
    y_pred = np.clip(y_pred, 1e-5, 1-1e-5) # избавляемся от нуля с помощью гистерезиса
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

def optimize(w, X, y, n_iterations, eta):
# потери будем записывать в список для отображения в виде графика
    losses = []
    for i in range(n_iterations):        
        loss, grad = log_loss(w, X, y)
        w = w - eta * grad
        losses.append(loss)
    return w, losses

def eval_model(X, y, iterations = 100, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        y_pred = sigmoid(W, X)
        err = calc_logloss(y, y_pred)
        W = W - alpha * (1/n * np.dot((y_pred - y), X.T))
    if i % (iterations / 10) == 0:
        print(i, W, err)
    return W

def calc_pred_proba(W, X):
    y_pred_proba = sigmoid(W, X)
    return y_pred_proba

def calc_pred(w, X, threshold = 0.5):
    m = X.shape[1]
    y_pred = np.zeros((1, m))
    w = w.reshape(X.shape[0], 1)
    
    A = sigmoid(w.T, X)
    for i in range(A.shape[1]):
        if (A[:,i] > threshold): 
            y_pred[:, i] = 1
        elif (A[:,i] <= threshold):
            y_pred[:, i] = 0
    return y_pred

# Метрики качевства
def accuracy(y, y_pred):
    return 100.0 - np.mean(np.abs(y - y_pred)*100.0)

def error_matrix(y, y_pred):
        m = np.zeros((2, 2))
        m[0,0] = np.sum(np.where(y == 1, y == y_pred, False))
        m[1,1] = np.sum(np.where(y == 0, y == y_pred, False))
        m[1,0] = np.sum(np.where(y == 1, y != y_pred, False))
        m[0,1] = np.sum(np.where(y == 0, y != y_pred, False))
        return m


In [23]:
# Стандартизация данных
x_st = X[1:,].copy()
x_mean, x_std = x_st.mean(axis=1).reshape(-1,1), x_st.std(axis=1).reshape(-1,1)
X[1:,] = (x_st - x_mean) / x_std

In [24]:
W = eval_model(X, y)

100 [ 0.49635335 -0.14077362  0.64537112  1.52180639] 0.7625324736043118


In [25]:
y_pred_proba = calc_pred_proba(W, X)
y_pred_proba

array([0.19476588, 0.23991495, 0.69245003, 0.21648985, 0.84461615,
       0.27492534, 0.97359525, 0.98239037, 0.1845406 , 0.75847462])

In [26]:
y_pred = calc_pred(W, X)
y_pred

array([[0., 0., 1., 0., 1., 0., 1., 1., 0., 1.]])

In [27]:
accuracy = accuracy(y, y_pred)
accuracy

80.0

In [38]:
matrix = error_matrix(y, y_pred)
matrix

array([[4., 1.],
       [1., 4.]])

In [43]:
precision = matrix[0,0] / (matrix[0,0] + matrix[0,1])
recall = matrix[0,0] / (matrix[0,0] + matrix[1,0])
precision, recall

(0.8, 0.8)

In [44]:
F1 = 2 * precision * recall / (precision + recall)
F1

0.8000000000000002